<a href="https://colab.research.google.com/github/yashkarkhanis01/Data_Science/blob/main/load_the_vehicle_data_for_Hillman_Ltd_into_the_data_vaul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import pandas as pd
import sqlite3 as sq
from pandas.io import sql
import uuid
pd.options.mode.chained_assignment = None
################################################################
Base = "VKHCG"
print("################################")
print("Working Base :", Base, " using ", sys.platform)
print("################################")
################################################################
Company = "03-Hillman"
InputDir = "00-RawData"
InputFileName = "VehicleData.csv"
################################################################
sDataBaseDir = Base + "/" + Company + "/03-Process/SQLite"
if not os.path.exists(sDataBaseDir):
  os.makedirs(sDataBaseDir)
################################################################
sDatabaseName = sDataBaseDir + "/Hillman.db"
conn1 = sq.connect(sDatabaseName)
################################################################
sDataVaultDir = Base + "/88-DV"
if not os.path.exists(sDataBaseDir):
  os.makedirs(sDataBaseDir)
################################################################
sDatabaseName = sDataVaultDir + "/datavault.db"
conn2 = sq.connect(sDatabaseName)
################################################################
sFileName = Base + "/" + Company + "/" + InputDir + "/" +InputFileName
print("###########")
print("Loading :", sFileName)
VehicleRaw = pd.read_csv(sFileName, header=0, low_memory=False, encoding="latin-1")
################################################################
sTable = "Process_Vehicles"
print("Storing :", sDatabaseName, " Table:", sTable)
VehicleRaw.to_sql(sTable, conn1, if_exists="replace")
################################################################
VehicleRawKey = VehicleRaw[["Make", "Model"]].copy()
VehicleKey = VehicleRawKey.drop_duplicates()
################################################################
VehicleKey["ObjectKey"] = VehicleKey.apply(lambda row: str("("+ str(row["Make"]).strip().replace(" ", "-").replace("/", "-").lower()+ ")-("+ (str(row["Model"]).strip().replace(" ", "-").replace(" ", "-").lower())+ ")"),axis=1,)
################################################################
VehicleKey["ObjectType"] = VehicleKey.apply(lambda row: "vehicle",axis=1)
################################################################
VehicleKey["ObjectUUID"] = VehicleKey.apply(lambda row:str(uuid.uuid4()), axis=1)
VehicleHub = VehicleKey[["ObjectType", "ObjectKey","ObjectUUID"]].copy()
VehicleHub.index.name = "ObjectHubID"
sTable = "Hub-Object-Vehicle"
print("Storing :", sDatabaseName, " Table:", sTable)
VehicleHub.to_sql(sTable, conn2, if_exists="replace")
################################################################
### Vehicle Satellite
################################################################
#
VehicleSatellite = VehicleKey[["ObjectType", "ObjectKey", "ObjectUUID", "Make", "Model"]].copy()
VehicleSatellite.index.name = "ObjectSatelliteID"
sTable = "Satellite-Object-Make-Model"
print("Storing :", sDatabaseName, " Table:", sTable)
VehicleSatellite.to_sql(sTable, conn2, if_exists="replace")
################################################################
### Vehicle Dimension
################################################################
sView = "Dim-Object"
print("Storing :", sDatabaseName, " View:", sView)
sSQL = "CREATE VIEW IF NOT EXISTS [" + sView + "] AS"
sSQL = sSQL + " SELECT DISTINCT"
sSQL = sSQL + " H.ObjectType,"
sSQL = sSQL + " H.ObjectKey AS VehicleKey,"
sSQL = sSQL + " TRIM(S.Make) AS VehicleMake,"
sSQL = sSQL + " TRIM(S.Model) AS VehicleModel"
sSQL = sSQL + " FROM"
sSQL = sSQL + " [Hub-Object-Vehicle] AS H"
sSQL = sSQL + " JOIN"
sSQL = sSQL + " [Satellite-Object-Make-Model] AS S"
sSQL = sSQL + " ON"
sSQL = sSQL + " H.ObjectType=S.ObjectType"
sSQL = sSQL + " AND"
sSQL = sSQL + " H.ObjectUUID=S.ObjectUUID;"
sql.execute(sSQL, conn2)
print("################")
print("Loading :", sDatabaseName, " Table:", sView)
sSQL = " SELECT DISTINCT"
sSQL = sSQL + " VehicleMake,"
sSQL = sSQL + " VehicleModel"
sSQL = sSQL + " FROM"
sSQL = sSQL + " [" + sView + "]"
sSQL = sSQL + " ORDER BY"
sSQL = sSQL + " VehicleMake"
sSQL = sSQL + " AND"
sSQL = sSQL + " VehicleMake;"
DimObjectData = pd.read_sql_query(sSQL, conn2)
DimObjectData.index.name = "ObjectDimID"
DimObjectData.sort_values(["VehicleMake", "VehicleModel"],inplace=True, ascending=True)
print("################")
print(DimObjectData)
#################################################################
print("################")
print("Vacuum Databases")
sSQL = "VACUUM;"
sql.execute(sSQL, conn1)
sql.execute(sSQL, conn2)
print("################")
#################################################################
conn1.close()
conn2.close()
#################################################################
print("### Done!! ############################################")
#################################################################

################################
Working Base : VKHCG  using  linux
################################
###########
Loading : VKHCG/03-Hillman/00-RawData/VehicleData.csv
Storing : VKHCG/88-DV/datavault.db  Table: Process_Vehicles
Storing : VKHCG/88-DV/datavault.db  Table: Hub-Object-Vehicle
Storing : VKHCG/88-DV/datavault.db  Table: Satellite-Object-Make-Model
Storing : VKHCG/88-DV/datavault.db  View: Dim-Object
################
Loading : VKHCG/88-DV/datavault.db  Table: Dim-Object
################
                  VehicleMake                       VehicleModel
ObjectDimID                                                     
2213               AM General                  DJ Po Vehicle 2WD
2212               AM General                   FJ8c Post Office
129                AM General                Post Office DJ5 2WD
131                AM General                Post Office DJ8 2WD
2869         ASC Incorporated                                GNX
...                       ...               